# Pré processamento da coluna SUMMARY

In [1]:
import pandas as pd
import string

In [2]:
try:
    df = pd.read_csv(
        '../data/new_tasks.csv',
        encoding='utf-8',
        sep=",",
    )
    print(f"CSV carregado com engine C! {len(df)} linhas encontradas.")

except Exception as e:
    print(f"Erro ao ler CSV: {e}")

CSV carregado com engine C! 4463 linhas encontradas.


In [3]:
# Verificar informações básicas do dataset
print("=== INFORMAÇÕES BÁSICAS DO DATASET ===")
print(f"Shape: {df.shape}")
print(f"Colunas: {df.columns.tolist()}")
print(f"Tipos de dados:\n{df.dtypes}")

# Informações apenas da coluna SUMMARY
print(f"\nResumo da coluna SUMMARY:")
print(f"  - Tipo de dado: {df['SUMMARY'].dtype}")
print(f"  - Valores únicos: {df['SUMMARY'].nunique()}")
print(f"  - Valores nulos: {df['SUMMARY'].isnull().sum()}")
print(f"  - Primeiros 5 valores:\n{df['SUMMARY'].head()}")

=== INFORMAÇÕES BÁSICAS DO DATASET ===
Shape: (4463, 6)
Colunas: ['SUMMARY', 'DTSTART', 'DTEND', 'CALENDAR', 'DURATION', 'CREATED']
Tipos de dados:
SUMMARY      object
DTSTART      object
DTEND        object
CALENDAR     object
DURATION    float64
CREATED      object
dtype: object

Resumo da coluna SUMMARY:
  - Tipo de dado: object
  - Valores únicos: 372
  - Valores nulos: 0
  - Primeiros 5 valores:
0     🍳
1     🧋
2    🍽️
3     🥪
4     🍌
Name: SUMMARY, dtype: object


### Análise

In [4]:
print("=== ANÁlISE DA COLUNA SUMMARY ===")

print(f"Quantidade de valores únicos: {df['SUMMARY'].nunique()}")
# print(f"Valores únicos: {df['SUMMARY'].unique()}")

# Exportar valores únicos da coluna SUMMARY para uma tabela CSV e usar uma LLM para clusterizar informacoes parecidas
unique_summary = df[['SUMMARY', 'CALENDAR']].drop_duplicates().rename(columns={'SUMMARY': 'TITLE'})
unique_summary = unique_summary.sort_values(by='TITLE').reset_index(drop=True)
unique_summary.to_csv('../data/title_calendar_unique.csv', index=False, encoding='utf-8')
print("Valores únicos exportados para ../data/title_unique.csv")


=== ANÁlISE DA COLUNA SUMMARY ===
Quantidade de valores únicos: 372
Valores únicos exportados para ../data/title_unique.csv


### Corrigindo os valores que estão com EMOJI

In [5]:
print(df['SUMMARY'].head(8))

# 🧋: lanche da manha
# 🍽️ e antes das 18:00: almoco
# 🥪: lanche da tarde
# 🍌: pre-treino
# 🍽️ e exatamente ou depois das 18:00: jantar

df.loc[df['SUMMARY'] == '🍳', 'SUMMARY'] = 'cafe da manha'
df.loc[df['SUMMARY'] == '🧋', 'SUMMARY'] = 'lanche da manha'
# Almoço: emoji 🍽️ e começa antes das 18h
df.loc[(df['SUMMARY'] == '🍽️') & (pd.to_datetime(df['DTSTART']).dt.hour < 18), 'SUMMARY'] = 'almoco'
df.loc[df['SUMMARY'] == '🥪', 'SUMMARY'] = 'lanche da tarde'
df.loc[df['SUMMARY'] == '🍌', 'SUMMARY'] = 'pre-treino'
df.loc[(df['SUMMARY'] == '🍽️') & (pd.to_datetime(df['DTSTART']).dt.hour >= 18), 'SUMMARY'] = 'jantar'

print(df['SUMMARY'].head(8))


0     🍳
1     🧋
2    🍽️
3     🥪
4     🍌
5    🍽️
6     🍳
7     🧋
Name: SUMMARY, dtype: object
0      cafe da manha
1    lanche da manha
2             almoco
3    lanche da tarde
4         pre-treino
5             jantar
6      cafe da manha
7    lanche da manha
Name: SUMMARY, dtype: object


## Limpando

In [6]:
def normalize_text(text):
    if pd.isnull(text):
        return ""
    # Lowercase
    text = text.lower()
    # Remove accents
    # text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra spaces
    text = ' '.join(text.split())
    return text

# Avaliação antes da normalização
print("=== Avaliação da coluna SUMMARY antes da normalização ===")
print(f"Valores únicos: {df['SUMMARY'].nunique()}")
print(f"Exemplos aleatórios: {df['SUMMARY'].drop_duplicates().sample(5, random_state=42).tolist()}")

df['SUMMARY_NORM'] = df['SUMMARY'].apply(normalize_text)

# Avaliação depois da normalização
print("\n=== Avaliação da coluna SUMMARY_NORM depois da normalização ===")
print(f"Valores únicos: {df['SUMMARY_NORM'].nunique()}")
print(f"Exemplos aleatórios: {df['SUMMARY'].drop_duplicates().sample(5, random_state=42).tolist()}")


=== Avaliação da coluna SUMMARY antes da normalização ===
Valores únicos: 373
Exemplos aleatórios: ['Pulo Big Jump', 'INF1301 - Relatório Plano de Ação', 'Zion', 'Niver Giovanna Lopes', 'Avaliação 4 Álgebra']

=== Avaliação da coluna SUMMARY_NORM depois da normalização ===
Valores únicos: 369
Exemplos aleatórios: ['Pulo Big Jump', 'INF1301 - Relatório Plano de Ação', 'Zion', 'Niver Giovanna Lopes', 'Avaliação 4 Álgebra']


### Dropando valores que não são necessários

In [7]:
print('Numero de linhas antes de retirar RescueTime:', len(df))
df = df[df['CALENDAR'] != 'RescueTime']
print('Numero de linhas depois de retirar RescueTime:', len(df))

Numero de linhas antes de retirar RescueTime: 4463
Numero de linhas depois de retirar RescueTime: 4418


In [8]:
print('Número de linhas antes de retirar o Rotina: ', len(df))
df = df[df['CALENDAR'] != 'Rotina']
print('Número de linhas depois de retirar o Rotina: ', len(df))

Número de linhas antes de retirar o Rotina:  4418
Número de linhas depois de retirar o Rotina:  4372


In [9]:
print('Número de linhas antes de retirar Parabéns!: ', len(df))
df = df[df['SUMMARY'] != 'Parabéns!']
print('Número de linhas depois de retirar Parabéns!: ', len(df))

Número de linhas antes de retirar Parabéns!:  4372
Número de linhas depois de retirar Parabéns!:  4348


### Tratando valores da coluna SUMMARY que possuem dois valores na coluna CALENDAR

In [10]:
# Coluna SUMMARY possui 30 valores que tem duas variações em 'CALENDAR'
print("=== TRATANDO Duplicatas da coluna SUMMARY ===")
df['CALENDAR'] = df['CALENDAR'].replace('PUC 👻', 'PUC')

# Corrigindo os valores duplicados de 'Academia'
df.loc[(df['SUMMARY'] == 'Academia') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Exercícios' # 2
print(f'Valores de CALENDAR para "Academia": {df.loc[df['SUMMARY'] == 'Academia', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'Aldeia'
df.loc[(df['SUMMARY'] == 'Aldeia') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "Aldeia": {df.loc[df['SUMMARY'] == 'Aldeia', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'Alessandra'
df.loc[(df['SUMMARY'] == 'Alessandra') & (df['CALENDAR'] == 'Compromisso'), 'CALENDAR'] = 'Saúde' # 2
print(f'Valores de CALENDAR para "Alessandra": {df.loc[df["SUMMARY"] == "Alessandra", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Aula'
df.loc[(df['SUMMARY'] == 'Aula') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'PUC' # 2
print(f'Valores de CALENDAR para "Aula": {df.loc[df['SUMMARY'] == 'Aula', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'BD: T2 G2'
df.loc[(df['SUMMARY'] == 'BD: T2 G2') & (df['CALENDAR'] == 'PUC'), 'CALENDAR'] = 'Exames' # 2
print(f'Valores de CALENDAR para "BD: T2 G2": {df.loc[df['SUMMARY'] == 'BD: T2 G2', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'BIRUSAMBA DE VERÃO'
df.loc[(df['SUMMARY'] == 'BIRUSAMBA DE VERÃO'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "BIRUSAMBA DE VERÃO": {df.loc[df['SUMMARY'] == 'BIRUSAMBA DE VERÃO', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de De-Para
df.loc[(df['SUMMARY'] == 'De-Para') & (df['CALENDAR'] == 'Exames'), 'CALENDAR'] = 'PUC' # 2
print(f'Valores de CALENDAR para "De-Para": {df.loc[df["SUMMARY"] == "De-Para", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Muay Thai'
df.loc[(df['SUMMARY'] == 'Muay Thai') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Exercícios' # 2
print(f'Valores de CALENDAR para "Muay Thai": {df.loc[df["SUMMARY"] == "Muay Thai", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'Psicóloga'
df.loc[(df['SUMMARY'] == 'Psicóloga') & (df['CALENDAR'] == 'Compromisso'), 'CALENDAR'] = 'Saúde' # 2
print(f'Valores de CALENDAR para "Psicóloga": {df.loc[df["SUMMARY"] == "Psicóloga", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'SO: P1' # 2
df.loc[(df['SUMMARY'] == 'SO: P1') & (df['CALENDAR'] == 'PUC'), 'CALENDAR'] = 'Exames' # 2
print(f'Valores de CALENDAR para "SO: P1": {df.loc[df["SUMMARY"] == "SO: P1", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'Itaipava'
df.loc[(df['SUMMARY'] == 'Itaipava') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "Itaipava": {df.loc[df["SUMMARY"] == "Itaipava", "CALENDAR"].unique()}')

=== TRATANDO Duplicatas da coluna SUMMARY ===
Valores de CALENDAR para "Academia": ['Exercícios']
Valores de CALENDAR para "Aldeia": ['Social']
Valores de CALENDAR para "Alessandra": ['Saúde']
Valores de CALENDAR para "Aula": ['PUC']
Valores de CALENDAR para "BD: T2 G2": ['Exames']
Valores de CALENDAR para "BIRUSAMBA DE VERÃO": ['Social']
Valores de CALENDAR para "De-Para": ['PUC']
Valores de CALENDAR para "Muay Thai": ['Exercícios']
Valores de CALENDAR para "Psicóloga": ['Saúde']
Valores de CALENDAR para "SO: P1": ['Exames']
Valores de CALENDAR para "Itaipava": ['Social']


In [11]:
print("=== ENCONTRO DE DUPLICATAS DA COLUNA SUMMARY ===")
duplicates = df[df.duplicated(subset='SUMMARY', keep=False)]
# Seleciona duplicatas de SUMMARY que aparecem em mais de um CALENDAR
duplicates_sum_cal = (
    duplicates[['SUMMARY', 'CALENDAR']]
    .drop_duplicates()
    .groupby('SUMMARY')
    .filter(lambda x: x['CALENDAR'].nunique() > 1)
    .sort_values(by=['SUMMARY', 'CALENDAR'], ascending=[True, True])
)
print(f"Quantidade de duplicatas encontradas: {duplicates_sum_cal.shape[0]}")
print(f"Duplicatas encontradas:\n{duplicates_sum_cal}")

=== ENCONTRO DE DUPLICATAS DA COLUNA SUMMARY ===
Quantidade de duplicatas encontradas: 0
Duplicatas encontradas:
Empty DataFrame
Columns: [SUMMARY, CALENDAR]
Index: []


## Verificando valores parecidos

### Passando o valor 'Alessandra' da coluna SUMMARY para 'Psicóloga'

In [12]:
print(f"Quantidade de valores únicos antes da unificação: {df['SUMMARY'].nunique()}")

print('=== TROCANDO DE "Alessandra" para "Psiquiatra" ===')
df.loc[df['SUMMARY'] == 'Alessandra', 'SUMMARY'] = 'Psiquiatra'

print(f"Quantidade de valores únicos depois da unificação: {df['SUMMARY'].nunique()}")


psi_values = df[df['SUMMARY'].str.lower().str.startswith('psi', na=False)]['SUMMARY'].unique()
print(f"\nValores que começam com 'psi' antes da unificação: {psi_values}")

print('=== TROCANDO DE "Psicóloga" para "Psicólogo" ===')
df.loc[df['SUMMARY'] == 'Psicóloga', 'SUMMARY'] = 'Psicólogo'

psi_values = df[df['SUMMARY'].str.lower().str.startswith('psi', na=False)]['SUMMARY'].unique()
print(f"Valores que começam com 'psi' depois da unificação: {psi_values}")


Quantidade de valores únicos antes da unificação: 327
=== TROCANDO DE "Alessandra" para "Psiquiatra" ===
Quantidade de valores únicos depois da unificação: 326

Valores que começam com 'psi' antes da unificação: ['Psiquiatra' 'Psicóloga' 'Psicólogo']
=== TROCANDO DE "Psicóloga" para "Psicólogo" ===
Valores que começam com 'psi' depois da unificação: ['Psiquiatra' 'Psicólogo']


## Melhorando valores com INF, MAT ou ADM no inicio

### Verificando todos os valores que possuem inf, mat ou adm no inicio

In [13]:
print('=== Avaliando valores que começam com "inf", "mat" ou "adm"')

inf_mat_values = df.loc[
    df['SUMMARY'].str.lower().str.startswith('adm', na=False) |
    df['SUMMARY'].str.lower().str.startswith('inf', na=False) |
    df['SUMMARY'].str.lower().str.startswith('mat', na=False),
    'SUMMARY'
]
print(f'Quantidade de valores únicos: {inf_mat_values.nunique()}')

print(f'Valores:\n{inf_mat_values.value_counts()}')

=== Avaliando valores que começam com "inf", "mat" ou "adm"
Quantidade de valores únicos: 33
Valores:
SUMMARY
INF1410 - Gerenc Proj Inf                     39
INF1608 - Análise Numérica                    38
ADM1019 - Intr. Finanças                      38
INF1629 - P. Eng. de Software                 38
MAT4162 - Cálculo II                          38
INF1771 - IA                                  36
INF1307 - Games                               36
INF1316 - SO                                  36
INF1403 - IHC                                 35
INF1022 - ALS                                 35
INF1350 - Prog Sis Reat                       34
INF1027 - Teste e Medição de Software         33
INF1036 - Prob Comp                           32
INF1640 - Redes de Computadores               31
INF1631 - Estruturas Discretas                31
INF1636 - Programação Orientada à Objetos     31
INF1028 - Projeto e Construção de Sistemas    30
INF1721 - AA                                  30
INF1032 

### Retirando INF2471 - Prob Comp

In [14]:
print('Número de linhas antes de retirar INF2471 - Prob Comp: ', len(df))
df = df[df['SUMMARY'] != 'INF2471 - Prob Comp']
print('Número de linhas depois de retirar INF2471 - Prob Comp: ', len(df))


Número de linhas antes de retirar INF2471 - Prob Comp:  4348
Número de linhas depois de retirar INF2471 - Prob Comp:  4346


### Retirando MAT1162 - CALCULO A VARIAS VARIAVEIS I

In [15]:
print('Número de linhas antes de retirar MAT1162 - CÁLCULO A VÁRIAS VARIÁVEIS I: ', len(df))
df = df[df['SUMMARY'] != 'MAT1162 - CÁLCULO A VÁRIAS VARIÁVEIS I']
print('Número de linhas depois de retirar MAT1162 - CÁLCULO A VÁRIAS VARIÁVEIS I: ', len(df))

Número de linhas antes de retirar MAT1162 - CÁLCULO A VÁRIAS VARIÁVEIS I:  4346
Número de linhas depois de retirar MAT1162 - CÁLCULO A VÁRIAS VARIÁVEIS I:  4343


In [16]:
# Corrigir primeiro código que já estava sendo tratado
df.loc[df['SUMMARY'] == 'INF1410 - Gerenc Proj Inf', 'SUMMARY'] = 'inf1410 - gerencia de projetos de informatica'

# INF1629 - P. Eng. de Software
df.loc[df['SUMMARY'] == 'INF1629 - P. Eng. de Software', 'SUMMARY'] = 'inf1629 - principios de engenharia de software'

# INF1314 - Startup IA
df.loc[df['SUMMARY'] == 'INF1314 - Startup IA', 'SUMMARY'] = 'inf1314 - gerando startup com inteligencia artificial'

# INF1771 - IA
df.loc[df['SUMMARY'] == 'INF1771 - IA', 'SUMMARY'] = 'inf1771 - inteligencia artificial'

# INF1307 - Games
df.loc[df['SUMMARY'] == 'INF1307 - Games', 'SUMMARY'] = 'inf1307 - desenvolvimento de jogos'

# INF1316 - SO
df.loc[df['SUMMARY'] == 'INF1316 - SO', 'SUMMARY'] = 'inf1316 - sistemas operacionais'

# INF1403 - IHC
df.loc[df['SUMMARY'] == 'INF1403 - IHC', 'SUMMARY'] = 'inf1403 - interacao humano computador'

# INF1022 - ALS
df.loc[df['SUMMARY'] == 'INF1022 - ALS', 'SUMMARY'] = 'inf1022 - algebra linear e sistemas'

# INF1350 - Prog Sis Reat
df.loc[df['SUMMARY'] == 'INF1350 - Prog Sis Reat', 'SUMMARY'] = 'inf1350 - programacao de sistemas reativos'

# INF1036 - Prob Comp
df.loc[df['SUMMARY'] == 'INF1036 - Prob Comp', 'SUMMARY'] = 'inf1036 - probabilidade computacional'

# INF1721 - AA
df.loc[df['SUMMARY'] == 'INF1721 - AA', 'SUMMARY'] = 'inf1721 - analise de algoritmos'

# INF1010 - EDA
df.loc[df['SUMMARY'] == 'INF1010 - EDA', 'SUMMARY'] = 'inf1010 - estruturas de dados avançadas'

# INF1338 - LLM
df.loc[df['SUMMARY'] == 'INF1338 - LLM', 'SUMMARY'] = 'inf1338 - large language models'

# INF1022 - Analisadores Léx. e Sint.
df.loc[df['SUMMARY'] == 'INF1022 - Analisadores Léx. e Sint.', 'SUMMARY'] = 'inf1022 - analisadores lexicos e sintaticos'

# INF2471 - Prob Comp
df.loc[df['SUMMARY'] == 'INF2471 - Prob Comp', 'SUMMARY'] = 'inf2471 - probabilidade computacional'

print('✅ Expansão de abreviações concluída - tudo em minúsculo!')

✅ Expansão de abreviações concluída - tudo em minúsculo!


### Verificando novamente os valores com INF, MAT ou ADM no inicio

In [17]:
print('=== Avaliando valores que começam com "inf", "mat" ou "adm"')

inf_mat_values = df.loc[
    df['SUMMARY'].str.lower().str.startswith('adm', na=False) |
    df['SUMMARY'].str.lower().str.startswith('inf', na=False) |
    df['SUMMARY'].str.lower().str.startswith('mat', na=False),
    'SUMMARY'
]
print(f'Quantidade de valores únicos: {inf_mat_values.nunique()}')

print(f'Valores:\n{inf_mat_values.value_counts()}')

=== Avaliando valores que começam com "inf", "mat" ou "adm"
Quantidade de valores únicos: 31
Valores:
SUMMARY
inf1410 - gerencia de projetos de informatica            39
INF1608 - Análise Numérica                               38
ADM1019 - Intr. Finanças                                 38
inf1629 - principios de engenharia de software           38
MAT4162 - Cálculo II                                     38
inf1771 - inteligencia artificial                        36
inf1316 - sistemas operacionais                          36
inf1307 - desenvolvimento de jogos                       36
inf1022 - algebra linear e sistemas                      35
inf1403 - interacao humano computador                    35
inf1350 - programacao de sistemas reativos               34
INF1027 - Teste e Medição de Software                    33
inf1036 - probabilidade computacional                    32
INF1636 - Programação Orientada à Objetos                31
INF1640 - Redes de Computadores                   

# Exportar DataFrame completo para CSV

In [18]:
# # Salvar dataset processado
print("=== SALVANDO DATASET PROCESSADO ===")

# Salvar como CSV processado
output_path = '../data/tasks_summary.csv'
df.to_csv(output_path, index=False, encoding='utf-8')
print(f"✅ Dataset processado salvo em: {output_path}")

# Estatísticas finais
print(f"\n=== ESTATÍSTICAS FINAIS ===")
print(f"Total de registros: {len(df)}")
print(f"Categorias únicas: {df['CALENDAR'].nunique()}")


=== SALVANDO DATASET PROCESSADO ===
✅ Dataset processado salvo em: ../data/tasks_summary.csv

=== ESTATÍSTICAS FINAIS ===
Total de registros: 4343
Categorias únicas: 11
